In [13]:
import pandas as pd
import numpy as np
import os
import time
from itertools import chain


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier


logit_smote  = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

logit_smote = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

svc_lin  = SVC(C=1.0, class_weight=None, coef0=0.0,
               decision_function_shape='ovr', degree=3, gamma=1.0, kernel='linear',
               max_iter=1000, probability=True, random_state=None, shrinking=True,
               tol=0.001, verbose=False)

svc_rbf  = SVC(C=1.0, kernel='rbf', 
               degree= 3, gamma=1.0, 
               coef0=0.0, shrinking=True,
               probability=True,tol=0.001,
               class_weight=None,
               verbose=False,max_iter= 1000,
               random_state=None)

xgc = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                    colsample_bytree=1, gamma=0, learning_rate=0.9, max_delta_step=0,
                    max_depth = 7, min_child_weight=1, missing=None, n_estimators=100,
                    n_jobs=-1, objective='binary:logistic', random_state=0,
                    reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                    silent=True, subsample=1)

gnb = GaussianNB(priors=None)

knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

rf = RandomForestClassifier()

dtc = DecisionTreeClassifier()


models = list()
models.append(('LR', logit_smote))
models.append(('SVM', svc_lin))
models.append(('SVM-rbf', svc_rbf))
models.append(('Gnb', gnb))
models.append(('RF', rf))
models.append(('DT', dtc))
models.append(('XGBoost', xgc))
models.append(('knn', knn))

# Datasets
dataset_name = ['Bank', 'C2C', 'DSN', 'HR', 'K2009', 'KKBox', 'Member', 'Mobile', 'SATO', 'TelC', 'TelE', 'UCI', 'news']
dataset_name = ['Bank']

# Sampling methods
l_sampling = ["plain", "smote", "adasyn", "tomekLinks", "ncr", "smoteRandom", "smoteTomek", "smoteNcr"]
l_sampling = ["smote"]

In [14]:
for ds_name in dataset_name:

    if ds_name == 'KKBox':
        target_name = 'churn'
        target_name_test = 'is_churn'
    else:
        target_name = 'churn'
        target_name_test = 'churn'

    fold_nb = 1
    for sampling in l_sampling:
        for fold_nb in range(1,6):
            dico = {}
            sep = ";"

            
            path_train = f"/home/ec2-user/SageMaker/data/churn_package/churn/sampled_datasets/{ds_name}/kfolds/{ds_name}_{sampling}_skf_k{fold_nb}_train.csv"
            path_test = f"/home/ec2-user/SageMaker/data/churn_package/churn/sampled_datasets/{ds_name}/kfolds/{ds_name}_{sampling}_skf_k{fold_nb}_test.csv"

            telcom_train = pd.read_csv(path_train, sep)        
            telcom_test = pd.read_csv(path_test, sep)  

            X_train, y_train = telcom_train.drop(target_name, axis=1), telcom_train[target_name]
            X_test, y_test = telcom_test.drop(target_name_test, axis=1), telcom_test[target_name_test]
            #y_test.rename(columns={'is_churn': 'churn'}, inplace=True)

            print(f"training set for {sampling} on fold {fold_nb} opened")
            print(f"test set for {sampling} on fold {fold_nb} opened")

            for name, model in models:
                # begin
                start = time.time()
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                # return an array:
                # first column = class 0, second column = class 1
                # [0.5162321 , 0.4837679 ],
                # [0.02658997, 0.97341003],
                y_pred_proba = model.predict_proba(X_test)

                # end
                end = time.time()
                running_time = round((end - start), 5)  
                    
                features = X_train.columns                 

                # for every single observation set name
                dico[f"name_{name}"] = list(chain(*([n]*len(y_test) for n in [name])))
                # 
                dico[f"y_{name}"] = y_test
                dico[f"yhat_{name}"] = y_pred
                dico[f"p(y=1)_{name}"] = np.round(y_pred_proba[:,1], decimals=4)
                dico[f"run_time_{name}"] = list(chain(*([n]*len(y_test) for n in [running_time])))

            path_out_test = f"churn_package/churn/sampled_datasets/{ds_name}/results/{ds_name}_{sampling}_skf_k{fold_nb}.csv"

            #_write_dataframe_to_csv_on_s3(pd.DataFrame(dico), path_out_test)

            print(f"fold number {fold_nb}")


training set for smote on fold 1 opened
test set for smote on fold 1 opened


/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting w

[17:01:17] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:01:17] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fold number 1
training set for smote on fold 2 opened
test set for smote on fold 2 opened


/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting w

[17:01:39] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:01:39] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fold number 2
training set for smote on fold 3 opened
test set for smote on fold 3 opened


...

[17:02:00] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:02:00] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fold number 3
training set for smote on fold 4 opened
test set for smote on fold 4 opened


/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting w

[17:02:22] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:02:22] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fold number 4
training set for smote on fold 5 opened
test set for smote on fold 5 opened


/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/louis_default/lib/python3.6/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting w

[17:02:43] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:02:43] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fold number 5
